Importing required libraries:

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
# from tensorflow.keras.models import load_model
# model = load_model("/content/drive/MyDrive/next_word.keras")


In [3]:
from google.colab import files
uploaded = files.upload()

Saving blue_castle.txt to blue_castle.txt


Load and Pre-Process the data

In [4]:
file = open("blue_castle.txt", "r", encoding = "utf8" )


lines = []
for i in file:
     lines.append(i)


data = ""
for i in lines:
    data = '  '. join(lines)


data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space


data = data.split()
data = ' '.join(data)
data[:500]


'The Project Gutenberg eBook of The Blue Castle, by Lucy Maud Montgomery This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before usi'

In [5]:
len(data)

402906

Apply tokenization and some other changes

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

#saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl','wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 112, 97, 587, 4, 1, 94, 147, 58, 2383, 2384, 1818, 51, 587, 42]

In [7]:
len(sequence_data)

72052

In [8]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

8413


In [9]:
sequences = []

for i in range (3, len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequences.append(words)

print("The Length if sequences are:" , len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length if sequences are: 72049


array([[   1,  112,   97,  587],
       [ 112,   97,  587,    4],
       [  97,  587,    4,    1],
       [ 587,    4,    1,   94],
       [   4,    1,   94,  147],
       [   1,   94,  147,   58],
       [  94,  147,   58, 2383],
       [ 147,   58, 2383, 2384],
       [  58, 2383, 2384, 1818],
       [2383, 2384, 1818,   51]])

In [10]:
X = []
y = []

for i in sequences:
  X.append(i[0:3])
  y.append(i[3])

X = np.array(X)
y = np.array(y)

In [11]:
print("Data:", X[:10])
print("Response:", y[:10])

Data: [[   1  112   97]
 [ 112   97  587]
 [  97  587    4]
 [ 587    4    1]
 [   4    1   94]
 [   1   94  147]
 [  94  147   58]
 [ 147   58 2383]
 [  58 2383 2384]
 [2383 2384 1818]]
Response: [ 587    4    1   94  147   58 2383 2384 1818   51]


In [12]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Creating the model

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation= "relu"))
model.add(Dense(vocab_size, activation="softmax"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
!pip install pydot graphviz


In [16]:
!apt-get -qq install -y graphviz && pip install pydot
import pydot

Plot the model

In [17]:

# from tensorflow import keras
# from keras.utils.vis_utils import plot_model

# keras.utils.plot_model(model, to_file='plot.png', show_layer_names=True)

Train the model

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

checkpoint = ModelCheckpoint("next_word.keras", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=5, batch_size=64, callbacks=[checkpoint])


Epoch 1/5
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 6.8261
Epoch 1: loss improved from inf to 6.69115, saving model to next_word.keras
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 33s 28ms/step - loss: 6.8259
Epoch 2/5
1125/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 6.2139
Epoch 2: loss improved from 6.69115 to 6.17375, saving model to next_word.keras
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 39s 26ms/step - loss: 6.2139
Epoch 3/5
1125/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.8415
Epoch 3: loss improved from 6.17375 to 5.81766, saving model to next_word.keras
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 31s 27ms/step - loss: 5.8414
Epoch 4/5
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.5714
Epoch 4: loss improved from 5.81766 to 5.54957, saving model to next_word.keras
1126/1126 ━━━━━━━━━━━━━━━━━━━━ 31s 27ms/step - loss: 5.5714
Epoch 5/5
1125/1126 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.3185
Epoch 5: loss improved from 5.54957 to 5.31761, saving model to next_word.keras
1126/1126 ━━━━━━━━━

Lets Predict

In [20]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

#Load the model and tokenizer
model = load_model('next_word.keras')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predict_word = ""

  for key, value in tokenizer.word_index.items():
    if value == preds:
      predicted_word = key
      break

  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line:")

  if text == "0":
    print("Execution completed....")
    break

  else:
    try:
      text = text.split(" ")
      text = text[-3:]
      print(text)

      Predict_Next_Words(model, tokenizer, text)


    except Exception as e:
       print("Error occured: ",e)
       continue

Enter your line:hello my name
['hello', 'my', 'name']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
she
Enter your line:hello mr
['hello', 'mr']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
benjamin


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# checkpoint = ModelCheckpoint("/content/drive/MyDrive/next_word.keras", monitor='loss', verbose=1, save_best_only=True)
# model.fit(X, y, epochs=20, batch_size=64, callbacks=[checkpoint])
